### Import Libraries

In [12]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from docx2pdf import convert
import pprint

### Extract information from text

In [18]:
def getInvoiceInfo(path):
    '''
    Get invoice info from pdf/doc for standard invoices 
    '''
    #Convert doc -> pdf
    if path.endswith('.doc'):
        path = 'output.pdf'
        convert('path', path)
        
    #init params
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    
    
    #Read pdf
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    
    #Text pre processing
    
    #Get all the items as list without any empty strings
    text = list(filter(None, text.split('\n')))
    
    #Text Features
    
    features = {}
    
    features['Invoice Number'] = text[text.index('Invoice Number') + 3]
    features['Due Date'] = text[text.index('Due Date') + 5]
    features['Rate/Price'] = text[text.index('Rate/Price') + 1]
    features['Total'] = text[text.index('Rate/Price') + 2]
    
    return features

In [19]:
pprint.pprint(getInvoiceInfo('sample.pdf'))

{'Due Date': 'January 31, 2016',
 'Invoice Number': 'Due Date',
 'Rate/Price': '$85.00',
 'Total': 'Adjust'}


<b>If the PDF contains images, we can use the model defined in the second notebook to extract text and process information</b>